<a href="https://colab.research.google.com/github/Anteii/HPC-Labs/blob/main/lab4/MassSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook setup

In [1]:
from IPython.display import clear_output

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
!git clone https://github.com/NVIDIA/cuda-samples.git
!make -C /content/cuda-samples/Samples/1_Utilities/deviceQueryDrv/
!/content/cuda-samples/bin/x86_64/linux/release/deviceQueryDrv

Cloning into 'cuda-samples'...
remote: Enumerating objects: 11024, done.
remote: Counting objects: 100% (11024/11024), done.
remote: Compressing objects: 100% (1839/1839), done.
remote: Total 11024 (delta 9184), reused 10979 (delta 9159), pack-reused 0
Receiving objects: 100% (11024/11024), 127.03 MiB | 27.16 MiB/s, done.
Resolving deltas: 100% (9184/9184), done.
Checking out files: 100% (3615/3615), done.
make: Entering directory '/content/cuda-samples/Samples/1_Utilities/deviceQueryDrv'
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common  -m64    --threads 0 --std=c++11 -gencode arch=compute_35,code=compute_35 -o deviceQueryDrv.o -c deviceQueryDrv.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=compute_35 -o deviceQueryDrv deviceQueryDrv.

In [5]:
!mkdir src

# Python (CPU)

In [150]:
import numpy as np
np.random.seed(41)

def generate_sub(alphabet, h_min, h_max):
  sub_len = np.random.randint(h_min, h_max, size=1)
  return np.random.choice(alphabet, size=sub_len)

def gen_d(alphabet, subs):
  d = {ch: [] for ch in alphabet}
  for sub_ind, sub in enumerate(subs):
    for ch_ind, ch in enumerate(sub):
      d[ch].append((sub_ind, ch_ind))
  return d

def gen_working_mat(subs, N, H):
  mat = np.array([len(sub) for sub in subs])
  return mat.reshape(-1,1).repeat(H, axis=1)

def iterate(text, d, mat):
  for ind, ch in enumerate(text):
    for pair in d[ch]:
      mat[pair[0], ind - pair[1]] -= 1
  return mat

def is_contain(mat):
  return (mat == 0).any(axis=1)

def find_indices(mat):
  sub_ind, pos_ind = np.where(mat == 0)
  return np.stack([sub_ind, pos_ind]).T

In [140]:
# Define alphabet
S = 6
alphabet = np.arange(0, S, dtype=np.uint8)
# Example
#S = 6
#alphabet = ["a", "b", "c", "d", "e", "f"]

In [141]:
# Define text
H = 100
text = np.random.choice(alphabet, size=H)

# Example
#H = 6
#text = np.array(["a", "a", "e", "f", "e", "d"])

In [142]:
h_min, h_max = 1, 20
N = 10
subs = [generate_sub(alphabet, h_min, h_max) for i in range(N)]

# Example
#N = 3
#subs = [np.array(["a", "a"]), np.array(["a"]), np.array(["f", "e", "d"])]

In [143]:
d = gen_d(alphabet, subs)
mat = gen_working_mat(subs, N, H)

In [144]:
mat = iterate(text, d, mat)

In [151]:
is_contain(mat)

array([False,  True, False, False,  True,  True, False, False, False,
        True])

In [ ]:
find_indices(mat)

# C++ (CPU)

In [ ]:
%%writefile src/cpu.cpp
#include <iostream>

using namespace std;

int main(){
  cout << "Hello World!" << endl;
  return 0;
}

Overwriting src/cpu.cpp


In [ ]:
!g++ -O2 src/cpu.cpp -o cpu

In [ ]:
!./cpu

Hello World!


# C++ (GPU)